In [2]:
import pyspark
from pyspark.sql import SparkSession
import pandas

In [3]:
spark = SparkSession.builder.appName("apple_compare").getOrCreate()
# sc = SparkContext(appName='mm_exp', conf=conf)

24/01/16 19:36:07 WARN Utils: Your hostname, arthurpfonseca resolves to a loopback address: 127.0.1.1; using 192.168.0.19 instead (on interface wlo1)
24/01/16 19:36:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/16 19:36:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
apple_df = spark.read.option("header", "true").csv("apple_quality.csv")
apple_df.show()

+----+------------+------------+------------+------------+------------+------------+------------+-------+
|A_id|        Size|      Weight|   Sweetness| Crunchiness|   Juiciness|    Ripeness|     Acidity|Quality|
+----+------------+------------+------------+------------+------------+------------+------------+-------+
|   0|-3.970048523|-2.512336381| 5.346329613|-1.012008712| 1.844900361| 0.329839797|-0.491590483|   good|
|   1|-1.195217191|-2.839256528| 3.664058758| 1.588232309| 0.853285795| 0.867530082|-0.722809367|   good|
|   2|-0.292023862|-1.351281995|-1.738429162|-0.342615928| 2.838635512|-0.038033328| 2.621636473|    bad|
|   3|-0.657195773|-2.271626609| 1.324873847|-0.097874716| 3.637970491|-3.413761338| 0.790723217|   good|
|   4|  1.36421682|-1.296611877|-0.384658206| -0.55300577| 3.030874354|-1.303849429| 0.501984036|   good|
|   5|-3.425399755|-1.409082204|-1.913511195|-0.555774864| -3.85307147| 1.914615916|-2.981523169|    bad|
|   6| 1.331605736| 1.635955715| 0.875974244| 

In [10]:
apple_df.select(["Size", "Weight"]).show()

+------------+------------+
|        Size|      Weight|
+------------+------------+
|-3.970048523|-2.512336381|
|-1.195217191|-2.839256528|
|-0.292023862|-1.351281995|
|-0.657195773|-2.271626609|
|  1.36421682|-1.296611877|
|-3.425399755|-1.409082204|
| 1.331605736| 1.635955715|
|-1.995462096| -0.42895848|
|-3.867632233|-3.734513576|
|-0.727982709|-0.442820353|
| -2.69933629|-1.329506988|
| 2.450959845|-0.564177407|
|-0.170811719|-1.867271142|
|-1.345530536|-1.623701121|
| 2.839580937|-0.344798192|
|-2.659887385|-2.795684208|
|-1.468951547|-1.950359588|
|-0.074370177|-4.714749953|
|-0.302364267| 1.724395847|
|-2.108049899| 0.356467403|
+------------+------------+
only showing top 20 rows



In [14]:
train_df, test_df = apple_df.randomSplit(weights=[0.8, 0.2], seed=42)
train_df.show()

+----+------------+------------+------------+------------+------------+------------+------------+-------+
|A_id|        Size|      Weight|   Sweetness| Crunchiness|   Juiciness|    Ripeness|     Acidity|Quality|
+----+------------+------------+------------+------------+------------+------------+------------+-------+
|   0|-3.970048523|-2.512336381| 5.346329613|-1.012008712| 1.844900361| 0.329839797|-0.491590483|   good|
|   1|-1.195217191|-2.839256528| 3.664058758| 1.588232309| 0.853285795| 0.867530082|-0.722809367|   good|
| 100|-1.581378996|-0.759501459| 0.135359641| 1.881819633| 1.998052361| 0.729362217| 0.286883321|   good|
|1000|-1.450722623|-1.873303787|-0.743177804| 1.981552145|-2.092821414| 1.691483862| 0.784607915|    bad|
|1001|-0.157271619| 0.829703853| -3.27215596| 1.226894151| -1.36458925| 1.487907926| 1.052510895|    bad|
|1003|  0.48573438|-4.694670684| -1.94594246| 2.022091675| 0.861829748|-0.643013894|-1.414626558|   good|
|1005|-2.025514845|-1.073743514| -0.56538837|-

In [52]:
from sklearn import svm
from sklearn import preprocessing

train_feature_df = train_df.select(["Size", "Weight", "Sweetness", "Crunchiness", "Juiciness", "Ripeness", "Acidity"])
train_predict_col_df = train_df.select(["Quality"])

X_train = [list(features) for features in train_feature_df.collect()]
y_train = [list(features) for features in train_predict_col_df.collect()]
encoder = preprocessing.LabelEncoder().fit(y_train)
y_train = encoder.transform(y_train)
y_train

/home/arthurpfonseca/Documents/Poli/5oQuadriAula/TCC/data-analysis/.venv/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/arthurpfonseca/Documents/Poli/5oQuadriAula/TCC/data-analysis/.venv/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


array([1, 1, 1, ..., 1, 0, 0])

In [50]:
clf = svm.SVC()
clf.fit(X_train, y_train)

SVC()

In [53]:
test_feature_df = test_df.select(["Size", "Weight", "Sweetness", "Crunchiness", "Juiciness", "Ripeness", "Acidity"])
test_predict_col_df = test_df.select(["Quality"])

X_test = [list(features) for features in test_feature_df.collect()]
y_test = [list(features) for features in test_predict_col_df.collect()]
y_test = encoder.transform(y_test)
y_test

/home/arthurpfonseca/Documents/Poli/5oQuadriAula/TCC/data-analysis/.venv/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


array([0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0,

In [55]:
y_predict = clf.predict(X_test)
encoder.inverse_transform(y_predict)

array(['bad', 'good', 'bad', 'good', 'good', 'good', 'good', 'good',
       'bad', 'good', 'bad', 'good', 'bad', 'good', 'bad', 'good', 'bad',
       'bad', 'good', 'good', 'bad', 'bad', 'bad', 'bad', 'bad', 'bad',
       'good', 'good', 'good', 'good', 'bad', 'good', 'good', 'bad',
       'bad', 'bad', 'bad', 'bad', 'bad', 'bad', 'good', 'bad', 'bad',
       'bad', 'good', 'good', 'bad', 'good', 'good', 'bad', 'good',
       'good', 'good', 'bad', 'bad', 'bad', 'good', 'good', 'bad', 'bad',
       'good', 'good', 'good', 'good', 'bad', 'good', 'good', 'bad',
       'bad', 'good', 'bad', 'bad', 'good', 'bad', 'bad', 'good', 'bad',
       'good', 'good', 'bad', 'good', 'good', 'good', 'good', 'bad',
       'good', 'bad', 'good', 'good', 'bad', 'bad', 'bad', 'bad', 'good',
       'good', 'bad', 'good', 'good', 'good', 'good', 'bad', 'bad',
       'good', 'good', 'bad', 'bad', 'bad', 'good', 'good', 'good', 'bad',
       'bad', 'good', 'bad', 'good', 'good', 'bad', 'bad', 'good', 'bad',
 

In [58]:
n_test_samples = len(y_test)
n_correct_predicts = 0
for i in range(n_test_samples):
	if y_predict[i] == y_test[i]:
		n_correct_predicts += 1

print("accuracy: ", (n_correct_predicts / n_test_samples))

accuracy:  0.8787061994609164


  Using cached scikit_learn-1.3.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached threadpoolctl-3.2.0-py3-none-any.whl.metadata (10.0 kB)
Using cached scikit_learn-1.3.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (10.8 MB)
Using cached threadpoolctl-3.2.0-py3-none-any.whl (15 kB)

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [11]:
# from pyspark.mllib.linalg import SparseVector
# from pyspark.mllib.classification import SVMWithSGD

# # data = [
# #     LabeledPoint(0.0, [0.0]),
# #     LabeledPoint(1.0, [1.0]),
# #     LabeledPoint(1.0, [2.0]),
# #     LabeledPoint(1.0, [3.0])
# # ]

# data = [
#     (0.0, 0.0),
#     (1.0, 1.0),
#     (1.0, 2.0),
#     (1.0, 3.0)
# ]
# svm = SVMWithSGD.train(sc.parallelize(data), iterations=10)
# svm.predict([1.0])

# svm.predict(sc.parallelize([[1.0]])).collect()

# svm.clearThreshold()
# svm.predict(numpy.array([1.0]))

NameError: name 'LabeledPoint' is not defined